# Métodos Numéricos Aplicados à Transferência de Calor

## Introdução

O normal do nosso fluxo de trabalho é iniciar importando as bibliotecas que vamos utilizar no decorrer do material.
Um maior detalhamento sobre elas já foi feito na aula anterior, de modo que agora podemos utilizar diretamente:

In [ ]:
import handcalcs.render
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pint
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import scipy as sp
import scipy.integrate
import scipy.optimize
from tqdm.notebook import tqdm

px.defaults.template = "ggplot2"
px.defaults.height = 600
pio.templates.default = "ggplot2"

ureg = pint.UnitRegistry()

O bloco a seguir é opcional, ele vai alterar o estilo padrão de nossas figuras, e aumentar um pouco o seu tamanho, melhorando a apresentação em nossa aula:

In [ ]:
# Definindo um novo estilo para as figuras [opcional]
# Esse bloco modifica alguns dos valores padrões para

plt.rcdefaults()

# https://matplotlib.org/3.1.0/gallery/style_sheets/style_sheets_reference.html
plt.style.use("ggplot")

# https://matplotlib.org/3.1.1/tutorials/introductory/customizing.html
plt.rcParams.update({"figure.dpi": 100, "figure.figsize": (6, 6)})

px.defaults.template = "ggplot2"
px.defaults.height = 600
pio.templates.default = "ggplot2"

## Exercícios Resolvidos

### Radiação e convecção combinadas em transferência de calor permanente unidirecional

A superfície interior de uma parede de espessura $L=0,25m$ é mantida a $21^oC$, enquanto a temperatura no meio externo é $T_\infty = 4^oC$. Considere que há troca de calor com a vizinhança $T_{viz} = 250 K$, o coeficiente de convecção é $h=23W/m^2\cdot ^oC$, a condutividade térmica do material que compõe a parede é $k=0,65 W/m \cdot ^oC$, a emissividade da superfície externa vale $\epsilon = 0,8$ e a constante de Stefan-Boltzmann $\sigma = 5,67 \times 10^{-8} [W/m^2 \cdot K^4]$. Determine a temperatura externa da parede $T_2$.

<img src="../Assets/radiacao.png">

\begin{equation}
    k \left( \dfrac{T_1-T_2}{L} \right) = \epsilon \sigma \left( T_2^4 - T_{viz}^4 \right) + h \left( T_2 - T_\infty \right)
\end{equation}

* **Resolução**:

Como primeiro passo para resolver o problema proposto, vamos atribuir todos os parâmetros do problema:

In [ ]:
%%render

L = ( 0.25 * ureg("meter") ) # Espessura da parede
T_1 = ( ureg.Quantity(21.0, ureg.degC).to("K") ) # Temperatura parede interior
T_inf = ( ureg.Quantity(4.0, ureg.degC).to("K") ) # Temperatura no meio externo
T_viz = ( 250.0 * ureg("K") ) # Temperatura da vizinhança
h = ( 23.0 * ureg("W/(m**2*K)") ) # Coeficiente de convecção
k = ( 0.65 * ureg("W/(m*K)") ) # Condutividade térmica do material da parede
epsilon = ( 0.8 ) # Emissividade da superfície externa
sigma = ( 5.68e-8 * ureg("W/(m**2*K**4)") ) # Constante de Stefan-Boltzmann

No segundo passo, reescrevemos a equação do problema deixando o lado esquerdo zerado, para que possamos programar e obter o resultado numérico para o lado direito:

\begin{equation}
    0 = \epsilon \sigma \left( T_2^4 - T_{viz}^4 \right) + h \left( T_2 - T_\infty \right) - k \left( \dfrac{T_1-T_2}{L} \right) = f(T_2)
\end{equation}

In [ ]:
def equação_exemplo_1(T_2: float) -> float:
    T_2 *= ureg("K")
    
    return (
        epsilon * sigma * (T_2 ** 4.0 - T_viz ** 4.0)
        + h * (T_2 - T_inf)
        - k * (T_1 - T_2) / L
    )

Temos um caso de "atingir a meta", qual o valor de $T_2$ que zera a nossa função?

O que fazemos agora é resolver esse problema numericamente, de maneira interativa. Podemos fazer suposições sobre o valor de $T_2$ até satisfazer a equação, ou seja, o retorno da função acima deve ser próximo de zero.

Fisicamente, faz sentido que $T_{viz} < T_2 < T_1$, então vamos começar com esses palpites iniciais:

In [ ]:
equação_exemplo_1(T_2 = 300.0)

In [ ]:
equação_exemplo_1(T_2 = 250.0)

Ambos valores com sinais opostos nos são um grande indicativo, pois existe de fato uma raiz da função nesse intervalo!

Podemos ir fazendo novos palpites em intervalos cada vez menores, até encontrar a raiz da função. Nosso próximo passo seria:

In [ ]:
equação_exemplo_1(T_2 = 275.0)

Vamos esclarecer isso com a representação gráfica:

In [ ]:
T_2 = np.linspace(start = 250.0, stop = 300.0, num = 201)

resultados_T_2 = equação_exemplo_1(T_2)

fig = px.line(x=T_2, y=resultados_T_2, labels = {"x": fr"$T_2 [K]$", "y": fr"$f(T_2) [W/m^2]$"})
fig.show()

Mas observe que Python conta com um leque enorme de ferramentas à nossa disposição, de modo que não precisamos programar todo esse processo, podemos por exemplo usar a função [`fsolve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fsolve.html) do pacote SciPy (lembre-se a acessar a documentação com `sp.optimize.fsolve?` se você precisar de ajuda.

Veja o código:

In [ ]:
T_2 = sp.optimize.fsolve(equação_exemplo_1, x0 = T_1) * ureg("K")
T_2

* **Resposta:**

A aproveitando as facilidades para o gerenciamento de unidades, vamos converter o resultado para graus Celsius:

In [ ]:
T_2.to("degC")

### Condução de calor transiente bidimensional

Uma placa de cobre de $50cm \times 50cm$ inicialmente possui temperatura em toda a sua extensão igual a $0^oC$. Instantaneamente, suas bordas são levadas às temperaturas de $60^oC$ em $x=0$; $20^oC$ em $x=50cm$; $0^oC$ em $y=0$ e $100^oC$ em $y=50$. A difusividade térmica do cobre é $1,1532cm^2/s$. Considerando um $\Delta t = 4s$, $\Delta x = \Delta y = 5cm$, calcule a evolução da temperatura para a posição central da placa até o tempo de $400s$.
    Equação bidimensional:
\begin{equation}
\alpha \left( \dfrac{\partial ^2 T}{\partial x^2} + \dfrac{\partial ^2 T}{\partial y^2} \right) =\dfrac{\partial T}{\partial t}, \quad 0 \le x \le L_x, \quad 0 \le y \le L_y, \quad t \ge 0,
\end{equation}

\begin{equation}
T(x=0,y) = T_a,
\end{equation}
\begin{equation}
T(x=L_x,y) = T_b,
\end{equation}
\begin{equation}
T(x,y=0) = T_c,
\end{equation}
\begin{equation}
T(x,y=Ly) = T_d,
\end{equation}
\begin{equation}
T(x,y) = T_i, \quad para \quad t=0.
\end{equation}

Lembre-se que o critério de estabilidade numérica do problema é:
\begin{equation}
\Delta t \le \dfrac{\Delta x^2}{4 \alpha}.
\end{equation}

Discretizando com a derivada segunda numa representação por diferença central e a derivada primeira com diferença ascendente:

\begin{equation}
\dfrac{T^{n+1}_{i,j}-T^{n}_{i,j}}{\Delta t}=\alpha \left[ \dfrac{T^{n}_{i-1,j}-2T^{n}_{i,j}+T^{n}_{i+1,j}}{(\Delta x)^2} +\dfrac{T^{n}_{i,j-1}-2T^{n}_{i,j}+T^{n}_{i,j+1}}{(\Delta y)^2}  \right], \quad 1 \le i \le I - 2, \quad 1 \le j \le J - 2, \quad n > 0,
\end{equation}

Agora devemos isolar a incógnita do nosso problema: o termo $T_{i,j}^{n+1}$. Perceba que todos os termos à direita são conhecidos, e usamos essa informação para avançar progressivamente no tempo:

\begin{equation}
T^{n+1}_{i,j} = T^{n}_{i,j} + \alpha \Delta t \left[ \dfrac{T^{n}_{i-1,j}-2T^{n}_{i,j}+T^{n}_{i+1,j}}{(\Delta x)^2} +\dfrac{T^{n}_{i,j-1}-2T^{n}_{i,j}+T^{n}_{i,j+1}}{(\Delta y)^2}  \right], \quad 1 \le i \le I - 2, \quad 1 \le j \le J - 2, \quad n \ge 0.
\end{equation}

Veja como ficou o código:

In [ ]:
def equação_exemplo_2(x, y, t, alpha, T_a, T_b, T_c, T_d, T_i):
    # Condição inicial
    T = T_i * np.ones((x.size, y.size, t.size), order="F")

    # Condições de contorno
    T[0, :, :], T[-1, :, :], T[:, 0, :], T[:, -1, :] = T_a, T_b, T_c, T_d
    
    # Passo de tempo e resolução da malha ao quadrado
    dt, dx2, dy2 = t[1] - t[0], (x[1] - x[0]) ** 2.0, (y[1] - y[0]) ** 2.0

    # Estabilidade numérica
    print(f"dt = {dt}; dx2/(4*alpha) = {dx2/(4.*alpha)}")

    # Aqui resolve-se a equação
    for n in tqdm(range(0, t.size - 1)):
        for i in range(1, x.size - 1):
            for j in range(1, y.size - 1):
                
                T[i, j, n + 1] = T[i, j, n] + alpha * dt * (
                    (T[i-1, j, n] - 2 * T[i, j, n] + T[i+1, j, n]) / dx2
                    + (T[i, j-1, n] - 2 * T[i, j, n] + T[i, j+1, n]) / dy2
                )
    
    return T

Finalmente, podemos resolver o problema e graficar a solução:

In [ ]:
x = np.linspace(start=0.0, stop=50.0, num=11) * ureg.cm
y = np.linspace(start=0.0, stop=50.0, num=11) * ureg.cm
t = np.linspace(start=0.0, stop=400.0, num=101) * ureg.seconds

T = equação_exemplo_2(
    x,
    y,
    t,
    alpha = 1.1532 * ureg("cm**2/s"),
    T_a = ureg.Quantity(60.0, ureg.degC).to("K"),
    T_b = ureg.Quantity(20.0, ureg.degC).to("K"),
    T_c = ureg.Quantity(0.0, ureg.degC).to("K"),
    T_d = ureg.Quantity(100.0, ureg.degC).to("K"),
    T_i = ureg.Quantity(0.0, ureg.degC).to("K")
).to("degC")

In [ ]:
fig = px.imshow(
    T,
    animation_frame=-1,
    x=x,
    y=y[::-1],
    color_continuous_scale='RdBu_r',
    title=f"Temperatura - {T.units}",
    labels=dict(x=f"x - {x.units}", y=f"y - {x.units}", animation_frame="snapshot"),
    origin = "lower"
)
fig.show()

In [ ]:
fig = px.line(
    x = t,
    y = T[T.shape[0]//2,T.shape[1]//2,:],
    title=f"Temperatura no centro da placa",
    labels=dict(x=f"tempo - {t.units}", y=f"T - {T.units}")
)
fig.show()

#### Condução permanente com geração de calor

Considere uma placa de urânio de espessura $L = 4cm$ e condutividade térmica $k = 28 W/m \cdot K$ em que o calor é gerado uniformemente a uma taxa constante de $\dot{e} = 5 \times 10^6 W/m^3$. Um dos lados da placa é mantido a $0^oC$ com água gelada, enquanto o outro lado está sujeito à convecção para o ambiente a $T_\infty = 30^oC$, com o coeficiente de transferência de calor $h=45 W/m^2 \cdot K$. Considerando o total de três pontos igualmente espaçados no meio, dois nos contornos e um no centro, estime a temperatura da superfície exposta da placa sob condições permanentes usando a abordagem de diferenças finitas.

<img src="../Assets/uranio.png">

Escrevendo a equações para cada nó:
\begin{equation}
T_0 = 0 \\
\dfrac{T_0 - 2 T_1 + T_2}{\Delta x^2} + \dfrac{\dot{e}_1}{k} = 0 \\
h(T_\infty - T_2) + k\dfrac{T_1 - T_2}{\Delta x} + \dot{e}(\Delta x/2) = 0
\end{equation}

O problema pode ser reescrito na forma matricial como:
\begin{equation}
\begin{split}
\underbrace{
\begin{bmatrix}
1 & 0 & 0 \\
1/\Delta x^2 & -2/\Delta x^2 & 1/\Delta x^2 \\
0 & k/\Delta x & -h-k/\Delta x
\end{bmatrix}
}_{A}
\begin{bmatrix}
T_{0} \\ T_{1} \\ T_{2}
\end{bmatrix}
=
\underbrace{
\begin{bmatrix} 0 \\ -\dot{e}/k \\ -\dot{e}\Delta x/2 -hT_\infty \end{bmatrix}
}_{B}
\end{split}
\end{equation}

In [ ]:
def equação_exemplo_2(L, k, eg, T_inf, h):

    x = np.linspace(0.0, L, num=3, endpoint=True)
    dx = x[1] - x[0]

    A = np.array(
        [
            [1.0, 0.0, 0.0],
            [1.0 / dx ** 2.0, -2.0 / dx ** 2.0, 1.0 / dx ** 2.0],
            [0.0, k / dx, -h - k / dx],
        ]
    )
    B = np.array([0.0, -eg / k, -eg * (dx / 2.0) - h * T_inf])

    # Solver de álgebra linear do pacote numpy
    T = np.linalg.solve(A, B)
    
    return T

In [ ]:
L = 0.04  # Espessura da placa
k = 28.0 # Condutividade térmica do material
eg = 5.0e6 # Taxa de geração de calor
T_inf = 30.0  # Temperatura do ambiente
h = 45.0 # Coeficiente de transferência de calor por convecção

T = equação_exemplo_2(L, k, eg, T_inf, h)

In [ ]:
# Escrevendo o resultado na tela
for i, t in enumerate(T):
    print(f"T{i} = {t:5.2f}")

Esse problema pode também ser resolvido analiticamente como:
\begin{equation}
    T(x) = \dfrac{0,5 \dot{e}hL^2/k + \dot{e}L + T_\infty h}{hL + k}x - \dfrac{\dot{e}x^2}{2k}
\end{equation}

In [ ]:
def T_exata(x):
    return (
        x * (0.5 * eg * h * L ** 2.0 / k + eg * L + T_inf * h) / (h * L + k)
        - 0.5 * eg * x ** 2.0 / k
    )

O sistema matricial pode ser escrito de maneira genérica para uma decomposição em $n$ pontos, como:
\begin{equation}
\begin{split}
\begin{bmatrix}
1 & 0 & 0 \\
& \ddots & \ddots & \ddots \\
& & 1/\Delta x^2 & -2/\Delta x^2 & 1/\Delta x^2 \\
& & & \ddots & \ddots & \ddots \\
& & & 0 & k/\Delta x & -h-k/\Delta x
\end{bmatrix}
\begin{bmatrix}
T_{0} \\ \vdots \\ T_{i} \\ \vdots \\ T_{n}
\end{bmatrix}
=
\begin{bmatrix} 0 \\ \vdots \\ -\dot{e}_i/k \\ \vdots \\ -\dot{e}_n\Delta x/2 -hT_\infty \end{bmatrix}
\\
\mbox{com $1\leq i \leq n - 2$,}
\end{split}
\end{equation}

Vamos programar a solução acima de maneira genérica, agora em função do número de pontos `n`, seja só:

In [ ]:
def equação_exemplo_3(L, k, eg, T_inf, h, n):

    x = np.linspace(0.0, L, num=n, endpoint=True)
    dx = x[1] - x[0]

    A = np.zeros((n,n), dtype=np.float64)
    
    A[0,0] = 1.0
    
    for i in range(1, x.size - 1):
        A[i,i-1] = 1.0 / dx ** 2.0
        A[i,i] = - 2.0 / dx ** 2.0
        A[i,i+1] = 1.0 / dx ** 2.0
        
    A[-1, -2], A[-1, -1] = k / dx, -h - k / dx

    B = np.ones((n), dtype=np.float64) * (-eg / k)
    
    B[0] = 0.0
    B[-1] = -eg * (dx / 2.0) - h * T_inf

    # Solver de álgebra linear do pacote numpy
    T = np.linalg.solve(A, B)
    
    return x, T

Agora vamos testar a convergência da solução, para três valores de n:

In [ ]:
fig = go.Figure()

# Solução exata
x_exato = np.linspace(0., L, num=101)
fig.add_trace(
    go.Scatter(
        x=x_exato, y=T_exata(x_exato),
        mode='lines',
        name='Analítica'
    )
)

for n in [3, 5, 10]:
    x, T = equação_exemplo_3(L, k, eg, T_inf, h, n)

    fig.add_trace(
        go.Scatter(
            x=x, y=T,
            mode='lines+markers',
            name=f'n={n}'
        )
    )  

fig.show()

#### Armazenamento de energia solar em paredes de Trombe

As paredes de alvenaria pintadas de cor escura, chamadas paredes de Trombe, são comumente usadas do lado sul das casas com energia solar passiva para absorver energia solar, armazenálas durante o dia e liberá-la para a casa durante a noite (mais informações [aqui](https://en.wikipedia.org/wiki/Trombe_wall)). Normalmente uma camada única ou dupla de vidro é colocada fora da parede e transmite a maior parte da energia solar, bloqueando as perdas de calor da superfície exposta da parede externa. Além disso, saídas de ar são comumente instaladas nas partes inferior e superior entre a parede de Trombe e os vidros. Em sua passagem, o ar tem a temperatura elevada e entra na sala através da abertura no lado superior da parece.

<img src="../Assets/casa_1.png" width="300">

Considere uma parede Trombe com $30cm$ de espessura, condutividade térmica de $k=0,69W/m\cdot K$ e difusividade térmica $\alpha = 4,44 \times 10^-7 m^2/s$. A variação de temperatura ambiente $T_{sai}$ e do fluxo de calor solar $\dot{q}_{solar}$ inidente sobre a face vertical da superfície virada ao Sul ao longo de um dia típico de janeiro é dada por

|Período do dia| Temperatura ambiente, $^oC$|Radiação solar, $W/m^2$|
|--|--|--|
|7h-10h|0,6|360|
|10h-13|6,1|763|
|13h-16h|7,2|562|
|16h-19h|2,8|0|
|19h-22h|0|0|
|22h-1h|-2,8|0|
|1h-4h|-3,3|0|
|4h-7h|-3,9|0|

A parede de Trombe tem vidro simples com um produto de absortividade e transmissividade de $ \kappa = 0,77$ (ou seja, 77% da energia solar incidente é absorvida pela superfície exposta da parede de Trombe), e o coeficiente médio de transferência de calor combinada para perda de calor da parede de Trombe no ambiente é determinada a $h_{sai} = 4 W/m^2 \cdot C$. O interior da casa é mantido a $T_{ent} = 21 ^oC$ todo o tempo, e o coeficiente de transferência de calor na superfície interna da parede de Trombe é $h_{ent} = 10 W/m^2 \cdot ^oC$. Além disso, as aberturas na parede são mantidas fechadas, portanto a transferência de calor entre o ar da casa e a parede de Trombe ocorre apenas através da superfície interior da parede. Pressupondo que a temperatura da parede de Trombe varie linearmente entre $21 ^oC$ na superfície interna e $-1 ^oC$ na superfície externa às 7 h da manhã e usando o método explícito das diferenças finitas com espaçamento nodal uniforme de $\Delta x = 6 cm$, determine a distribuição de temperatura ao longo da espessura da parede de Trombe após 12, 24, 36 e 48 h. Além disso, determine a quantidade líquida de calor transferido da parede de Trombe para a casa durante o primeiro e o segundo dia. Considere que a parede tem $3m$ de altura e $7,5m$ de comprimento e utilize um passo de tempo de $\Delta t = 15min$. 

In [ ]:
table = pd.DataFrame(
    {
        "ti": [7.0, 10.0, 13.0, 16.0, 19.0, 22.0, 1.0, 4.0],
        "tf": [10.0, 13.0, 16.0, 19.0, 22.0, 1.0, 4.0, 7.0],
        "Tsai": [0.6, 6.1, 7.2, 2.8, 0.0, -2.8, -3.3, -3.9],
        "qsol": [360.0, 763.0, 562.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    }
)

<img src="../Assets/casa_0.png" width="300">

Escrevendo a equações para cada nó:
\begin{equation}
T_0^{j+1} = \left( 1 - 2\tau -2 \tau \dfrac{h_{ent}\Delta x}{k} \right) T_0^{j} + 2\tau T_1^{j} + 2\tau \dfrac{h_{ent}\Delta x}{k} T_{ent} \\
T_i^{j+1} = \tau (T_{i-1}^j + T_{i+1}^j) + (1-2\tau)T_{i}^j\\
T_n^{j+1} = \left( 1 - 2\tau -2 \tau \dfrac{h_{sai}\Delta x}{k} \right) T_n^{j} + 2\tau T_{n-1}^{j} + 2\tau \dfrac{h_{sai}\Delta x}{k} T_{sai}^j + 2\tau \dfrac{\kappa \dot{q}_{solar}^j \Delta x}{k}
\end{equation}
onde

\begin{equation}
\tau = \dfrac{\alpha \Delta t}{\Delta x^2}
\end{equation}

O problema pode ser reescrito na forma matricial como:
\begin{equation}
\begin{split}
\begin{bmatrix}
T_{0}^{j+1} \\ \vdots \\ T_{i}^{j+1} \\ \vdots \\ T_{n-1}^{j+1}
\end{bmatrix}
=
\underbrace{
\begin{bmatrix}
1 - 2\tau -2 \tau \dfrac{h_{ent}\Delta x}{k} & 2\tau & 0 \\
\ddots & \ddots & \ddots \\
\tau & 1-2\tau & \tau \\
\ddots & \ddots & \ddots \\
0 & 2\tau & 1 - 2\tau -2 \tau \dfrac{h_{sai}\Delta x}{k}
\end{bmatrix}
}_{A}
\begin{bmatrix}
T_{0}^j \\ \vdots \\ T_{i}^j \\ \vdots \\ T_{n-1}^j
\end{bmatrix}
+
\underbrace{
\begin{bmatrix} -2\tau \dfrac{h_{ent}\Delta x}{k} T_{ent} \\ \vdots \\ 0 \\ \vdots \\ - 2\tau \dfrac{h_{sai}\Delta x}{k} T_{sai}^j - 2\tau \dfrac{\kappa \dot{q}_{solar}^j \Delta x}{k} \end{bmatrix}
}_{B}
\\
\mbox{com $1 \leq i \leq n - 2$,}
\end{split}
\end{equation}

O critério de estabilidade para esse problema é dado por:
\begin{equation}
\Delta t \le \dfrac{\Delta x^2}{3,74 \alpha} = 36,13min
\end{equation}

In [ ]:
def equação_exemplo_4(L, dx, dth, k, alpha, kappa, hsai, Tent, hent, table):

    x = np.arange(0, L + dx, dx)
    time = np.arange(0.0, 48.0 + dth, dth)

    T = np.zeros((x.size, time.size))
    
    # convertendo para segundos
    dt = dth * 3600.0
    
    tau = alpha * dt / dx ** 2.0
    
    # Condição Inicial
    T[:, 0] = 21.0 - (22.0 / L) * x
    time += 7.0
    
    # Construção dos Operadores
    A = np.zeros((x.size,x.size))
    
    for i in range(1, x.size - 1):
        A[i,i-1] = tau
        A[i,i] = 1.0 - 2.0 * tau
        A[i,i+1] = tau
    
    B = np.zeros_like(x)

    # Laço de avanço temporal
    for j in tqdm(range(time.size - 1)):
        #
        # Obter os valores do contorno à partir da tabela
        #
        index = int((time[j] - 7.0) % 24.0 // 3)
        Tsai = table.loc[index].Tsai
        qsol = table.loc[index].qsol
        #
        # Condição de contorno em x=0
        #
        A[0, 0] = 1.0 - 2 * tau - 2.0 * tau * hent * dx / k
        A[0, 1] = 2.0 * tau
        A[0, 2] = 0.0
        B[0] = 2.0 * tau * hent * dx * Tent / k
        #
        # Condição de contorno em x=L
        #
        A[-1, -3] = 0.0
        A[-1, -2] = 2.0 * tau
        A[-1, -1] = 1.0 + -2.0 * tau - 2.0 * tau * hsai * dx / k
        B[-1] = 2.0 * tau * hsai * dx * Tsai / k + 2 * tau * kappa * qsol * dx / k
        #
        T[:, j + 1] = A.dot(T[:, j]) + B
    
    return x, time, T

In [ ]:
x, time, T = equação_exemplo_4(
    L = 0.3,  # m
    dx = 0.06,  # m
    dth = 0.25,  # h
    k = 0.69,  # W/mK
    alpha = 4.44e-7,  # m^2/s
    kappa = 0.77,
    hsai = 4,  # W/m^2K
    Tent = 21.0,  # graus C
    hent = 10,  # W/m^2K
    table = table
)

* Visualizando os resultados:

In [ ]:
fig = px.imshow(
    T.T,
    x=x,
    y=time,
    color_continuous_scale='RdBu_r',
    title=r"$\text{Temperatura} [^oC]$",
    labels=dict(x="x [m]", y="Tempo [h]", color="T"),
    aspect = "auto",
    origin = "lower"
)
fig.show()

* Variação da temperatura em três pontos da parede, em função do tempo

In [ ]:
fig = go.Figure()

for i in [0, x.size//2, -1]:

    fig.add_trace(
        go.Scatter(
            x=time, y=T[i,:],
            mode='lines',
            name=f'x={x[i]}'
        )
    )  

fig.show()

-----

> **Felipe N. Schuch**,<br>
> Pesquisador em Fluidodinâmica Computacional na PUCRS, com interesse em: Escoamentos turbulentos, transferência de calor e massa, e interação fluido-estrutura; Processamento e visualização de dados em Python; Jupyter Notebook como uma ferramenta de colaboração, pesquisa e ensino.<br>
> [felipeschuch@outlook.com](mailto:felipeschuch@outlook.com "Email") [@fschuch](https://twitter.com/fschuch "Twitter") [Aprenda.py](https://fschuch.github.io/aprenda.py "Blog") [@aprenda.py](https://www.instagram.com/aprenda.py/ "Instagram")<br>

-----